In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                title = html.xpath('//span[@itemprop="name"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_vehicle = html.xpath('//div[@id="country3-head"]/following-sibling::div[@class="pro_tab_container"]/descendant::td/text()')
                for vehicle in list_vehicle:
                    vehicle = vehicle.strip()
                    if vehicle != '':
                        year = int(vehicle.split(' ')[0])
                        make_model_engine = ' '.join(vehicle.split(' ')[1:])
                        if make_model_engine not in dict_vehicle:
                            dict_vehicle[make_model_engine] = [year]
                        else:
                            dict_vehicle[make_model_engine].append(year)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = html.xpath('//div[@id="country3-head"]/following-sibling::div[@class="pro_tab_container"]/descendant::div[@class="accordion_body"]/ul/li/text()')
                for vehicle in list_vehicle:
                    vehicle = vehicle.strip()
                    if vehicle != '' and vehicle != '...and more':
                        year = int(vehicle.split(' ')[0])
                        make_model_engine = ' '.join(vehicle.split(' ')[1:])
                        if make_model_engine not in dict_vehicle:
                            dict_vehicle[make_model_engine] = [year]
                        else:
                            dict_vehicle[make_model_engine].append(year)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make_model_engine in dict_vehicle:
                    year_max = max(dict_vehicle[make_model_engine])
                    year_min = min(dict_vehicle[make_model_engine])
                    if year_max == year_min:
                        list_vehicle.append(make_model_engine + ' ' + str(year_min))
                    else:
                        list_vehicle.append(make_model_engine + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@itemprop="image"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="country2-head"]/following-sibling::div[@class="pro_tab_container"]/table/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Title': title,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])
                
                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()] = ' '.join([text.strip() for text in list_tr[1].xpath('./td[2]/descendant-or-self::*/text()')]).strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1

https://www.turbochargerpros.com/buynow/40-30928_gr  <->  [ok] - 剩余数量：0 - 当前时间：17:38:59

爬虫输出中

爬虫结束
